In [1]:
!pip install transformers datasets sacrebleu rouge-score evaluate sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.9 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=5fc58edfb131dfaa62e4f152ed2fec431211ba88ac69a8ff2b90f74fcc65517e
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import os, shutil, json, evaluate, torch
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
    MarianTokenizer, MarianMTModel,
    Seq2SeqTrainingArguments, Seq2SeqTrainer,
    DataCollatorForSeq2Seq, ProgressCallback
)

2025-05-30 04:02:48.015380: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748577768.202569      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748577768.256231      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# # gg colab
# BASE = "/content/drive/MyDrive/NLP_final/"
# RAW_DIR = os.path.join(BASE, "data", "raw")
# TOKENIZER_DIR = os.path.join(BASE, "marian_ft", "tokenizer")
# OUTPUT = os.path.join(BASE, "marian_ft", "pretrained_mt")
# os.makedirs(OUTPUT, exist_ok=True)

# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# print("Device:", DEVICE)

In [5]:
# kaggle
BASE      = "/kaggle/input/final-nlp/NLP_final"
RAW_DIR   = os.path.join(BASE, "data", "raw")        # chứa train.csv, val.csv, test.csv
OUTPUT = os.path.join("/kaggle/working/marian_ft", "pretrained_mt")     # thư mục lưu model fine-tuned
os.makedirs(OUTPUT, exist_ok=True)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

Device: cuda


In [6]:
data_files = {spl : os.path.join(RAW_DIR, f"{spl}.csv")
              for spl in ["train","val","test"]}
raw_ds = load_dataset("csv", data_files=data_files, cache_dir="/content/cache")
print(raw_ds)

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['en', 'vi', 'len_en_words', 'len_vi_words'],
        num_rows: 105685
    })
    val: Dataset({
        features: ['en', 'vi', 'len_en_words', 'len_vi_words'],
        num_rows: 13210
    })
    test: Dataset({
        features: ['en', 'vi', 'len_en_words', 'len_vi_words'],
        num_rows: 13211
    })
})


In [7]:
MODEL_ID = "Helsinki-NLP/opus-mt-en-vi"   # Marian đã fine-tune sẵn
tokenizer = MarianTokenizer.from_pretrained(MODEL_ID)
model     = MarianMTModel.from_pretrained(MODEL_ID).to(DEVICE)
print("Loaded vocabulary =", tokenizer.vocab_size)


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/756k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/289M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Loaded vocabulary = 53685


In [8]:
MAX_LEN_SRC = 128          # EN
MAX_LEN_TGT = 128          # VI

def preprocess(batch):
    # ---------- encode EN (source) ----------
    model_inputs = tokenizer(
        batch["en"],
        max_length=MAX_LEN_SRC,
        truncation=True,
        padding="max_length",
    )

    # ---------- encode VI (target) ----------
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["vi"],
            max_length=MAX_LEN_TGT,
            truncation=True,
            padding="max_length",
        )

    # seq-2-seq Trainer cần:
    #   input_ids, attention_mask  – cho encoder
    #   labels                    – cho decoder
    model_inputs["labels"] = labels["input_ids"]
    model_inputs["decoder_attention_mask"] = labels["attention_mask"]

    return model_inputs


# raw_ds là DatasetDict chứa train/val/test
tok_ds = raw_ds.map(
    preprocess,
    batched=True,
    remove_columns=raw_ds["train"].column_names,   # bỏ cột 'en','vi', ...
    desc="Tokenising",
)

tok_ds.set_format(type="torch")   # tensors ready for DataLoader / Trainer
print(tok_ds)

model.safetensors:   0%|          | 0.00/289M [00:00<?, ?B/s]

Tokenising:   0%|          | 0/105685 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Tokenising:   0%|          | 0/13210 [00:00<?, ? examples/s]

Tokenising:   0%|          | 0/13211 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
        num_rows: 105685
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
        num_rows: 13210
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
        num_rows: 13211
    })
})


# Định nghĩa metrics và Trainer

In [9]:
bleu  = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")

def compute_metrics(results):
    preds = tokenizer.batch_decode(results.predictions, skip_special_tokens=True)
    refs  = tokenizer.batch_decode(results.label_ids, skip_special_tokens=True)
    bleu_s  = bleu.compute(predictions=preds, references=[[r] for r in refs])["score"]
    rouge1  = rouge.compute(predictions=preds, references=refs)["rouge1"]*100
    rouge2  = rouge.compute(predictions=preds, references=refs)["rouge2"]*100
    rougel  = rouge.compute(predictions=preds, references=refs)["rougeL"]*100
    return {"bleu": bleu_s, "rouge1": rouge1, "rouge2": rouge2, "rougeL": rougel}

In [10]:
BATCH = 8

args = Seq2SeqTrainingArguments(
    output_dir          = OUTPUT,
    per_device_train_batch_size = BATCH,
    per_device_eval_batch_size  = BATCH,
    learning_rate       = 5e-5,
    num_train_epochs    = 3,
    predict_with_generate = True,
    eval_strategy = "epoch",
    save_strategy       = "epoch",
    logging_steps       = 1000,
    load_best_model_at_end = True,
    metric_for_best_model  = "bleu",
    greater_is_better      = True,
    fp16 = torch.cuda.is_available(),
    report_to=[]
)

trainer = Seq2SeqTrainer(
    model           = model,
    args            = args,
    train_dataset   = tok_ds["train"],
    eval_dataset    = tok_ds["val"],
    tokenizer       = tokenizer,
    data_collator   = DataCollatorForSeq2Seq(tokenizer, model),
    compute_metrics = compute_metrics,
    callbacks       = [ProgressCallback]
)

/tmp/ipykernel_35/3823456764.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [11]:
trainer.train()
trainer.save_model(OUTPUT)
tokenizer.save_pretrained(OUTPUT)

  0%|          | 0/19818 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel
1,0.315400,0.292178,33.907576,74.154676,53.322948,66.764912
2,0.277500,0.288144,34.596238,74.256556,53.578936,66.953541
3,0.254400,0.287992,34.681965,74.344863,53.681072,67.078207


{'loss': 0.3515, 'grad_norm': 52285.3515625, 'learning_rate': 4.747956403269755e-05, 'epoch': 0.15}
{'loss': 0.3251, 'grad_norm': 57439.95703125, 'learning_rate': 4.4956605106468866e-05, 'epoch': 0.3}
{'loss': 0.3206, 'grad_norm': 44365.2421875, 'learning_rate': 4.243364618024019e-05, 'epoch': 0.45}
{'loss': 0.3216, 'grad_norm': 42953.203125, 'learning_rate': 3.991068725401151e-05, 'epoch': 0.61}
{'loss': 0.3194, 'grad_norm': 43031.09375, 'learning_rate': 3.738772832778282e-05, 'epoch': 0.76}
{'loss': 0.3154, 'grad_norm': 61441.70703125, 'learning_rate': 3.486476940155414e-05, 'epoch': 0.91}


  0%|          | 0/826 [00:00<?, ?it/s]

...

  0%|          | 0/826 [00:00<?, ?it/s]

{'eval_loss': 0.28799161314964294, 'eval_bleu': 34.681964977363904, 'eval_rouge1': 74.34486258591357, 'eval_rouge2': 53.681071754599195, 'eval_rougeL': 67.07820727192582, 'eval_runtime': 821.1559, 'eval_samples_per_second': 16.087, 'eval_steps_per_second': 1.006, 'epoch': 3.0}


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


{'train_runtime': 7390.7384, 'train_samples_per_second': 42.899, 'train_steps_per_second': 2.681, 'train_loss': 0.28603565686886406, 'epoch': 3.0}


('/kaggle/working/marian_ft/pretrained_mt/tokenizer_config.json',
 '/kaggle/working/marian_ft/pretrained_mt/special_tokens_map.json',
 '/kaggle/working/marian_ft/pretrained_mt/vocab.json',
 '/kaggle/working/marian_ft/pretrained_mt/source.spm',
 '/kaggle/working/marian_ft/pretrained_mt/target.spm',
 '/kaggle/working/marian_ft/pretrained_mt/added_tokens.json')

In [12]:
from torch.utils.data import DataLoader
BATCH = 8

val_loader = DataLoader(
    tok_ds["val"].with_format("torch"),
    batch_size = BATCH,
    shuffle    = False,
)
test_loader = DataLoader(tok_ds["test"].with_format("torch"), batch_size=8, shuffle=False)

In [13]:
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

def marian_translate_batch(loader, model, tokenizer, device, num_beams=1, max_len=128):
    model.eval()
    preds, refs = [], []
    for batch in tqdm(loader, desc="Translating", unit="batch"):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            gen_ids = model.generate(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=max_len,
                num_beams=num_beams,
                early_stopping=True
            )
        preds.extend(tokenizer.batch_decode(gen_ids, skip_special_tokens=True))
        labs = batch["labels"].clone()
        labs[labs == -100] = tokenizer.pad_token_id
        refs.extend(tokenizer.batch_decode(labs, skip_special_tokens=True))
    return preds, refs

preds, refs = marian_translate_batch(test_loader, model, tokenizer, DEVICE, num_beams=1, max_len=128)

# In điểm BLEU và ROUGE-1/2/L
bleu_score = bleu.compute(predictions=preds, references=[[r] for r in refs])['score']
rouge_scores = rouge.compute(predictions=preds, references=refs)

print("BLEU      :", round(bleu_score, 2))
print("ROUGE-1   :", round(rouge_scores['rouge1']*100, 2))
print("ROUGE-2   :", round(rouge_scores['rouge2']*100, 2))
print("ROUGE-L   :", round(rouge_scores['rougeL']*100, 2))

Translating:   0%|          | 0/1652 [00:00<?, ?batch/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:679: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


BLEU      : 34.39
ROUGE-1   : 73.88
ROUGE-2   : 52.88
ROUGE-L   : 66.56


In [19]:
def marian_translate(texts,
                     model=model,
                     tokenizer=tokenizer,
                     device=DEVICE,
                     num_beams=1,
                     max_len=128,
                     clean=True):
    """
    texts   : str  | list[str] – câu/ các câu Tiếng Anh
    returns : str  | list[str] – câu/ các câu Tiếng Việt
    """
    if isinstance(texts, str):
        single = True
        texts  = [texts]
    else:
        single = False

    # 1) Tokenize hàng loạt -> chuyển sang device
    batch = tokenizer(texts,
                      return_tensors="pt",
                      padding=True,
                      truncation=True,
                      max_length=max_len).to(device)

    # 2) Generate
    with torch.no_grad():
        gen_ids = model.generate(
            input_ids      = batch["input_ids"],
            attention_mask = batch["attention_mask"],
            max_length     = max_len,
            num_beams      = num_beams,
            early_stopping = True
        )

    # 3) Decode
    outs = tokenizer.batch_decode(gen_ids,
                                  skip_special_tokens=True)

    # 4) (optional) làm sạch dấu cách trước dấu câu
    if clean:
        outs = [s.replace(" .", ".")
                  .replace(" ,", ",")
                  .replace(" !", "!")
                  .replace(" ?", "?")
                for s in outs]

    return outs[0] if single else outs


In [20]:

# Nhiều câu
en_batch = [
    "I want to go to sleep",
     "Good morning! Did you sleep well?",
    "Yesterday the stock market plunged almost five percent.",
    "She wonders whether artificial intelligence will ever surpass human creativity.",
    "Please, turn off the lights before you leave the laboratory.",
    "Although it rained heavily, the concert continued until midnight.",
    "The report, which was published in 2023, estimates that global CO₂ emissions hit 37.4 gigatons.",
    "Have you ever tried Vietnamese egg coffee?",
    "If we fail to act now, future generations will pay the price.",
    "NASA's James Webb telescope recently captured breathtaking images of distant galaxies.",
    "In my opinion, learning a new language is like opening an extra window on the world."
]
vi_batch = marian_translate(en_batch, num_beams=5)
print("\n--- Sample Translations ---")
for en, vi in zip(en_batch, vi_batch):
    print(f"EN: {en}")
    print(f"VI: {vi}\n")


--- Sample Translations ---
EN: I want to go to sleep
VI: Tôi muốn đi ngủ.

EN: Good morning! Did you sleep well?
VI: Chúc buổi sáng tốt lành! Bạn có ngủ ngon không?

EN: Yesterday the stock market plunged almost five percent.
VI: Ngày hôm qua thị trường chứng khoán đã giảm gần 5 %.

EN: She wonders whether artificial intelligence will ever surpass human creativity.
VI: Cô tự hỏi liệu trí tuệ nhân tạo có vượt qua được sự sáng tạo của con người hay không.

EN: Please, turn off the lights before you leave the laboratory.
VI: Làm ơn tắt đèn đi trước khi bạn rời phòng thí nghiệm.

EN: Although it rained heavily, the concert continued until midnight.
VI: Mặc dù trời mưa nhiều, buổi hoà nhạc vẫn tiếp tục cho đến nửa đêm.

EN: The report, which was published in 2023, estimates that global CO₂ emissions hit 37.4 gigatons.
VI: Báo cáo, xuất bản năm 2023, ước tính lượng lượng khí thải CO2 toàn cầu đã giảm 37.4 tỉ tấn.

EN: Have you ever tried Vietnamese egg coffee?
VI: Bạn đã bao giờ thử cà phê

In [16]:
# zip_path = os.path.join(BASE, "marian_ft_deploy.zip")
# shutil.make_archive(zip_path.replace('.zip', ''), "zip", OUTPUT)
# print(f"Đã tạo file zip model cho deploy: {zip_path}")

In [17]:
# %%bash
# # ===== Cell 1: zip inference package =====
# EXP_DIR=/kaggle/working/marian_ft          # ← đổi nếu khác
# PKG_NAME=marian_ft_infer.zip               # tên file zip sẽ tạo

# cd "$EXP_DIR"/..

# zip -r "$PKG_NAME" \
#     marian_ft/model.safetensors \
#     marian_ft/config.json \
#     marian_ft/generation_config.json \
#     marian_ft/source.spm marian_ft/target.spm \
#     marian_ft/vocab.json \
#     marian_ft/tokenizer_config.json \
#     marian_ft/special_tokens_map.json

# echo "✅  Đã tạo: $(pwd)/$PKG_NAME"
# ls -lh "$PKG_NAME"

In [18]:
# # ==== Cell 3: Load lại ZIP của MarianFT (inference) và test bản dịch ====
# import os, zipfile, tempfile
# import torch
# from transformers import MarianTokenizer, MarianMTModel

# # 1) Đường dẫn tới file ZIP bạn đã tạo (trong /kaggle/working)
# ZIP_PATH = "/kaggle/working/marian_ft_infer.zip"

# # 2) Giải nén vào thư mục tạm
# tmpdir = tempfile.mkdtemp()
# with zipfile.ZipFile(ZIP_PATH, "r") as z:
#     z.extractall(tmpdir)

# # 3) Tự động tìm sub-folder chứa config.json/tokenizer files
# model_dir = None
# for root, dirs, files in os.walk(tmpdir):
#     if "config.json" in files and "tokenizer_config.json" in files:
#         model_dir = root
#         break
# if model_dir is None:
#     model_dir = tmpdir  # fallback

# print(f"→ Loading model/tokenizer from: {model_dir}")

# # 4) Load tokenizer + model
# tokenizer = MarianTokenizer.from_pretrained(model_dir)
# model     = MarianMTModel.from_pretrained(model_dir).to("cpu")  # or "cuda"

# # 5) Test dịch sample
# sent = "Hello, how are you?"
# inputs  = tokenizer(sent, return_tensors="pt")
# outputs = model.generate(**inputs, max_length=64)
# print("EN:", sent)
# print("VI:", tokenizer.decode(outputs[0], skip_special_tokens=True))
